# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [92]:
from azureml.core import Workspace, Experiment

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core import Workspace, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import Model
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.core import Run

import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [93]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
label = 'vfc_level2_3_name'

subscription_id = 'b968fb36-f06a-4c76-a15f-afab68ae7667'
resource_group = 'aml-quickstarts-201550'
workspace_name = 'quick-starts-ws-201550'

workspace = Workspace(subscription_id, resource_group, workspace_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [38]:
# Choose a name for your CPU cluster
cluster_name = "theresa"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
capstone_env = Environment.from_conda_specification(name='project-env', file_path='capstone_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#env = Environment(name="TheresaSampling")
capstone_env.register(workspace=workspace)

Found existing cluster, use it.

Running


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220504.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "pip",
                "python=3.6.2",
 

In [39]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = None #using Bayesian sampling. BanditPolicy(slack_factor=0.2, evaluation_interval=2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = {"--n_estimators": choice(100, 150, 300), "--min_samples_leaf": choice(3, 5)}

#TODO: Create your estimator and hyperdrive config
#estimator = <your estimator here>
sampling = BayesianParameterSampling(param_sampling, properties=None)


src = ScriptRunConfig(source_directory='.',
                        script='train.py',
                        compute_target=cpu_cluster,
                        environment=capstone_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=sampling,
                                max_total_runs=9,
                                max_concurrent_runs=3, 
                                primary_metric_name='f1_score',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [40]:
experiment_name = 'capstone_hyperdrive'
experiment=Experiment(workspace, experiment_name)
hyperdriverun = experiment.submit(config=hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [94]:
hyperdriverun = Run.get(workspace, 'HD_8f5bb2b3-14b9-49fd-91c3-996f0ee467ba')
RunDetails(hyperdriverun).show()

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [95]:
best_run = hyperdriverun.get_best_run_by_primary_metric()

model = best_run.register_model(model_name='randomforest', 
                           model_path='outputs/model/randomforest.pkl',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.19.1')
model.download(target_dir='.', exist_ok=True, exists_ok=None)

'randomforest.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [96]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

service_name = 'capstone-service-endpoint'

inference_config = InferenceConfig(entry_script='score.py', environment=capstone_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=workspace,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config,
                          overwrite=True)

service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-20 12:25:22+00:00 Creating Container Registry if not exists.
2022-07-20 12:25:22+00:00 Registering the environment.
2022-07-20 12:25:23+00:00 Use the existing image.
2022-07-20 12:25:23+00:00 Generating deployment configuration.
2022-07-20 12:25:24+00:00 Submitting deployment to compute.
2022-07-20 12:25:27+00:00 Checking the status of deployment capstone-service-endpoint..
2022-07-20 12:27:49+00:00 Checking the status of inference endpoint capstone-service-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [102]:
import numpy as np

test_data = np.zeros((1789, ))
print({"data": [test_data.tolist(), test_data.tolist()]})

{'data': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [103]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {"data": [test_data.tolist(), test_data.tolist()]}

body = str.encode(json.dumps(data))

url = 'http://73fe0ccc-f762-4702-9a98-864de09a8d6a.southcentralus.azurecontainer.io/score'

key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"


req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'["good", "good"]'


We can see here that the model returns good for the two vectors with all zeros.

In [104]:
print(result)

b'["good", "good"]'


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

